In [ ]:
# Imports

import torch
from torch import nn
import pytorch_lightning as pl
from src.b2bnet import OtkaDataModule

In [ ]:
# Model

class Autoencoder(pl.LightningModule):
    def __init__(self, n_features=8, hidden_size=8):
        super().__init__()

        self.hidden_size = hidden_size

        self.encoder = nn.LSTM(n_features, hidden_size, batch_first=True)
        self.decoder = nn.LSTM(hidden_size, n_features, batch_first=True)
        self.fc_decoder = nn.Linear(n_features, n_features)

    def forward(self, x):
        batch_size = x.size(0)
        n_timesteps = x.size(1)
        y_enc, (h_enc, c_enc) = self.encoder(x)
        x_enc = torch.rand(batch_size, n_timesteps, self.hidden_size)
        y_dec, (h_dec, c_dec) = self.decoder(x_enc, (h_enc, c_enc))
        y_hat = self.fc_decoder(y_dec)
        return y_dec

    def training_step(self, batch, batch_idx):
        X, subject_ids, y_b2b, y_cls = batch
        X_recon = self(X)
        loss = nn.functional.mse_loss(X_recon, X)
        self.log('train/loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        X, subject_ids, y_b2b, y_cls = batch
        X_recon = self(X)
        loss = nn.functional.mse_loss(X_recon, X)
        self.log('val/loss', loss)
        return loss

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-2)

In [ ]:
# Experiment

segment_size = 120
batch_size = 256
n_timesteps = 120*10  # 10 sec
n_features = 59
hidden_size = 59
max_epochs = 100

datamodule = OtkaDataModule(segment_size=segment_size, batch_size=batch_size)

model = Autoencoder(n_features=n_features, hidden_size=hidden_size)

trainer = pl.Trainer(max_epochs=max_epochs,accelerator='auto', log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)